In [2]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

# 1. 데이터 읽기 및 준비
file_path = './seeds_dataset.txt'
with open(file_path, 'r') as file:
    raw_data = file.read()

data = []
for line in raw_data.strip().split('\n'):
    values = [v for v in line.split('\t') if v.strip() != '']
    if len(values) == 8:  # seeds 데이터셋은 8개 값(7개 특성+1개 라벨)
        try:
            data.append(list(map(float, values)))
        except ValueError:
            continue
data = np.array(data)
print(data.shape)

/Users/kimdoyeon/Projects/SchoolProjects/AI-Study/week4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


(210, 8)


In [4]:
X = data[:, :-1]
y = data[:, -1].astype(int) - 1   # 클래스 0부터 시작

# 2. 데이터 3-way 분할 (Train/Validation/Test)
# 먼저 test set 분리(20%)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# train set에서 validation set 분리(20% of trainval)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval
)

print(f"Train: {X_train.shape[0]}, Validation: {X_val.shape[0]}, Test: {X_test.shape[0]}")

Train: 134, Validation: 34, Test: 42


In [6]:
# 3. 함수형 API
inputs = tf.keras.Input(shape=(X.shape[1],))

x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(32, activation='relu')(x)

outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model_func = tf.keras.Model(inputs=inputs, outputs=outputs)
model_func.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_func.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_val, y_val), verbose=0)

loss, acc = model_func.evaluate(X_test, y_test, verbose=0)

print(f'Functional API - Test accuracy: {acc:.4f}')

2025-05-27 11:45:55.744816: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-27 11:45:55.744883: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-27 11:45:55.744899: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-27 11:45:55.744975: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-27 11:45:55.744995: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-05-27 11:45:56.024264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Functional API - Test accuracy: 0.7857


In [8]:
## 교과서 코드
# inputs = Input(shape=(2,))
# x = Dense(2, activation='sigmoid')(inputs)
# prediction = Dense(1, activation='sigmoid')(x)

# model = Model(inputs=inputs, outputs=prediction)

# tmp = Dense(2, activation='sigmoid')
# x = tmp(inputs)

In [10]:
# 4. Sequential
model_seq = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model_seq.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_seq.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_val, y_val), verbose=0)

loss, acc = model_seq.evaluate(X_test, y_test, verbose=0)

print(f'Sequential - Test accuracy: {acc:.4f}')

/Users/kimdoyeon/Projects/SchoolProjects/AI-Study/week4/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Sequential - Test accuracy: 0.8095


In [12]:
# # 교과서 코드
# model_seq2 = sequential()
# model_seq2.add(Dense(units=2, input_shape=(2,), activation='sigmoid'))
# model_seq2.add(Dense(units=1, activation='sigmoid'))

In [14]:
# 5. Model 상속
class MyDNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(64, activation='relu')
        self.d2 = tf.keras.layers.Dense(32, activation='relu')
        self.d3 = tf.keras.layers.Dense(3, activation='softmax')
        
    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)
        
model_sub = MyDNN()
model_sub.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_sub.build(input_shape=(None, X.shape[1]))
model_sub.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_val, y_val), verbose=0)

loss, acc = model_sub.evaluate(X_test, y_test, verbose=0)

print(f'Model subclassing - Test accuracy: {acc:.4f}')

/Users/kimdoyeon/Projects/SchoolProjects/AI-Study/week4/.venv/lib/python3.9/site-packages/keras/src/layers/layer.py:395: UserWarning: `build()` was called on layer 'my_dnn', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model subclassing - Test accuracy: 0.7619


In [16]:
# ## 교과서 코드
# class SimpleMLP(Model):
#     def __init__(self, num_classes):
#         super(SimpleMLP, self).__init__(name='mlp')
#         self.num_classes = num_classes

#         self.dense1 = Dense(32, activation='sigmoid')
#         self.dense2 = Dense(num_classes, activation='sigmoid')

#     def call(self, inputs):
#         x = self.dense1(inputs)
#         return self.dense2(x)
# model = SimpleMLP()
# model.compile(...)
# model.fit(...)

In [18]:
# 테스트셋에 대한 정밀도, 재현율, F1, 혼동행렬 출력
y_pred = np.argmax(model_func.predict(X_test), axis=1)
print("\n=== Functional API 성능 지표 ===")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro'):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

=== Functional API 성능 지표 ===
F1 Score: 0.7638
Precision: 0.8079
Recall: 0.7857
Confusion Matrix:
 [[ 6  2  6]
 [ 1 13  0]
 [ 0  0 14]]


In [20]:
y_pred = np.argmax(model_seq.predict(X_test), axis=1)
print("\n=== Sequential 성능 지표 ===")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro'):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

=== Sequential 성능 지표 ===
F1 Score: 0.7927
Precision: 0.8218
Recall: 0.8095
Confusion Matrix:
 [[ 7  3  4]
 [ 1 13  0]
 [ 0  0 14]]


In [26]:
y_pred = np.argmax(model_sub.predict(X_test), axis=1)
print("\n=== Model 상속 성능 지표 ===")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro'):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

=== Model 상속 성능 지표 ===
F1 Score: 0.7291
Precision: 0.7783
Recall: 0.7619
Confusion Matrix:
 [[ 5  4  5]
 [ 1 13  0]
 [ 0  0 14]]
